In [1]:
# 读取数据
import pandas as pd
train_labled = pd.read_csv('data/data23977/train.csv')
test = pd.read_csv('data/data23977/test_dataset.csv')
sub = pd.read_csv('data/data23977/submit_example.csv')

In [2]:
label = []
ncw_label = train_labled["ncw_label"].tolist()
fake_label = train_labled["fake_label"].tolist()
real_label = train_labled["real_label"].tolist()
for i in range(len(train_labled)):
    if ncw_label[i] == 1 and fake_label[i] == 0 and real_label[i] == 0:
        label.append(0)
    elif ncw_label[i] == 0 and fake_label[i] == 1 and real_label[i] == 0:
        label.append(1)
    elif ncw_label[i] == 0 and fake_label[i] == 0 and real_label[i] == 1:
        label.append(2)
train_labled["label"] = label

In [3]:
print(train_labled.shape)
print(test.shape)
print(train_labled.columns)

(49910, 10)
(2614, 6)
Index(['id', 'content', 'picture_lists', 'category', 'ncw_label', 'fake_label',
       'real_label', 'comment_2c', 'comment_all', 'label'],
      dtype='object')


In [4]:
train_labled.head(3)

,id,content,picture_lists,category,ncw_label,fake_label,real_label,comment_2c,comment_all,label
0,4028762460708675,回复新浪网友对博文【国家文物局限制鉴宝节目现场估价转】的评论：;;查看原文：,NaN,文体娱乐,0,0,1,NaN,NaN,2
1,4126670854660291,//分享网易新闻:《发生在昆明的火锅店老板“辱滇门”，云南人该愤怒还是羞愧》|发生在昆明.....,63ad082a189566eed7c4bb3e4bc55012.jpg,社会生活,0,0,1,NaN,NaN,2
2,3702441948299962,西宁城管围殴民警扬言要把警察打死|西宁城管围...,4986dc2a5f09a87c7af5dfc57d7775cd.jpg,社会生活,0,0,1,NaN,NaN,2


In [5]:
# 更新paddlehub
!pip install --upgrade paddlehub -i https://pypi.tuna.tsinghua.edu.cn/simple

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
Requirement already up-to-date: paddlehub in d:\rgzn\lib\site-packages (1.8.3)


In [7]:
# 划分验证集，保存格式  text[\t]label
from sklearn.model_selection import train_test_split

train_labled["content"] = train_labled["content"].astype(str)
train_labled["content"] = train_labled["content"].apply(lambda x: x.replace('\t',''))
train_labled["content"] = train_labled["content"].apply(lambda x: x.replace('\n',''))

train_labled["comment_all"] = train_labled["comment_all"].astype(str)
train_labled["comment_all"] = train_labled["comment_all"].apply(lambda x: x.replace('\t',''))
train_labled["comment_all"] = train_labled["comment_all"].apply(lambda x: x.replace('\n',''))

#train_labled = train_labled[['content', 'label']]
train_labled = train_labled[['content','comment_all', 'label']]
train, valid = train_test_split(train_labled, test_size=0.2, random_state=2020)
train.to_csv('data/data23977/train.txt', index=False, header=False, sep='\t')
valid.to_csv('data/data23977/valid.txt', index=False, header=False, sep='\t')

In [8]:
import csv
import io
with io.open('data/train.txt', "r", encoding="UTF-8") as file:
    reader = csv.reader(file, delimiter="\t", quotechar=None)
    for (i, line) in enumerate(reader):
        ncol = len(line)
        if ncol < 3:
            print(i)
            print(line)


In [9]:
train_labled.info(verbose=True,null_counts=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 49910 entries, 0 to 49909
Data columns (total 3 columns):
content        49910 non-null object
comment_all    49910 non-null object
label          49910 non-null int64
dtypes: int64(1), object(2)
memory usage: 1.1+ MB


In [10]:
# 自定义数据集
import os
import codecs
import csv

from paddlehub.dataset.base_nlp_dataset import BaseNLPDataset

class MyDataset(BaseNLPDataset):
    """DemoDataset"""
    def __init__(self):
        # 数据集存放位置
        self.dataset_dir ="data/data23977/"
        super(MyDataset, self).__init__(
            base_path=self.dataset_dir,
            train_file="train.txt",
            dev_file="valid.txt",
            test_file="valid.txt",
            train_file_with_header=False,
            dev_file_with_header=False,
            test_file_with_header=False,
            # 数据集类别集合
            label_list=["0", "1", "2"])

dataset = MyDataset()
for e in dataset.get_train_examples()[:4]:
    print("{}\t{}\t{}".format(e.guid, e.text_a, e.text_b,e.label))

0	终于来了！&#xe627; 王凯 首张个人音乐作品#王凯画外音# 重磅完整上线🎊网页链接 超豪华金曲阵容&#xe627; 蔡健雅&#xe627; 吴青峰 等全亚洲优秀音乐人鼎力支持打造品质上乘的概念唱片凯哥哥用细腻温柔的声线...展开全文c	特别棒特别有诚意的专辑，喜欢@王凯kkw;;喜欢这种声音很有磁性[米奇比心][米奇比心]听完了整张专辑，王凯的歌声和独白特别打动人心，可以看出他是用很专业很用心的态度来对待这张专辑[good]像一个广播剧，听完像看完一场电影一样整张专辑就是一个完整的故事，非常的喜欢💕用os+歌曲的形式串联起一个完整的音乐故事，这种设计太有画面感，太棒了！很棒的音乐专辑，像一部音乐剧，一个完整的故事。正在循环听，整张专辑，如同一部音乐剧，感谢优秀的制作团队，感谢歌手王凯，非常喜欢。有声有画面的音乐，会让人有身临其境的感觉，词好，曲好，解说的好！演员的画外音，歌手的内心戏，就这么缓缓地流进了我的心……[米奇比心][米奇比心]画外音让人见识了王凯除演员之外，作为歌手的魅力，他在用音乐讲述内心的故事，教给我们怎样在伤痛面前收获一份淡然和洒脱。非常有创意又心意满满的作品，仿佛一部音乐剧。王凯独特的嗓音故事感太强，让人不由自主沉醉其中@王凯kkw诚意满满的作品[米奇比心][米奇比心][米奇比心]王凯哥哥这张专辑真的很棒！王凯画外音超话棒极啦！从头听到尾，感觉像是读了一个故事！好好听[米奇比心][给你小心心]王凯超话音樂與OS，串起了故事，闡述了畫外音。@王凯kkw;;好好听[给你小心心]1[给你小心心]感谢诚意满满的凯哥，感谢优秀的制作团队，一张特别的专辑，好极了@王凯kkw透过《画外音》，脑中浮现出一个完整画面，不管是一个人还是一群人，从冷清到温暖，认识王凯就像认识了小小的太阳，很炙热，很深情。真好！@王凯kkw@王凯kkw首张个人音乐作品「画外音」1、intro（序曲）2、一个人也很好（os）3、一人份的孤独4、不是真的（os）5、心上的茧6、那你呢（os）7、还有什么不敢8、画外音（os）9、脊椎（os）10、k＇sradio11、第一段没问题了，是吗？12、小小的太阳13、outro持續）听过之后便不舍得退出，循环一整天！慢慢的一整张专辑听下来才能理解王凯想说的。[心]1[心]2[心][心]特别棒@王凯kkw;;听了几天了很喜欢淡淡的诉说


In [11]:
# 加载模型
import paddlehub as hub
module = hub.Module(name="ernie")

[2020-12-04 23:16:54,057] [    INFO] - Installing ernie module
[2020-12-04 23:16:54,318] [    INFO] - Module ernie already installed in C:\Users\96075\.paddlehub\modules\ernie


In [12]:
# 构建Reader
reader = hub.reader.ClassifyReader(
    dataset=dataset,
    vocab_path=module.get_vocab_path(),
    sp_model_path=module.get_spm_path(),
    word_dict_path=module.get_word_dict_path(),
    max_seq_len=150)

[2020-12-04 23:17:14,546] [    INFO] - Dataset label map = {'0': 0, '1': 1, '2': 2}


In [13]:
# finetune策略
strategy = hub.AdamWeightDecayStrategy(
    weight_decay=0.01,
    warmup_proportion=0.1,
    learning_rate=5e-5)

In [18]:
# 运行配置
config = hub.RunConfig(
    use_cuda=False,
    num_epoch=1,
    checkpoint_dir="model",
    batch_size=32,
    eval_interval=100,
    strategy=strategy)

[2020-12-04 23:21:16,284] [    INFO] - Checkpoint dir: model


In [19]:
# Finetune Task
inputs, outputs, program = module.context(
    trainable=True, max_seq_len=150)
# Use "pooled_output" for classification tasks on an entire sentence.
pooled_output = outputs["pooled_output"]
feed_list = [
    inputs["input_ids"].name,
    inputs["position_ids"].name,
    inputs["segment_ids"].name,
    inputs["input_mask"].name,
]
cls_task = hub.TextClassifierTask(
        data_reader=reader,
        feature=pooled_output,
        feed_list=feed_list,
        num_classes=dataset.num_labels,
        config=config,
        metrics_choices=["acc"])

!!! The CPU_NUM is not specified, you should set CPU_NUM in the environment variable list.
CPU_NUM indicates that how many CPUPlace are used in the current task.
And if this parameter are set as N (equal to the number of physical CPU core) the program may be faster.

export CPU_NUM=8 # for example, set CPU_NUM as number of physical CPU core which is 8.

!!! The default number of CPU_NUM=1.
[2020-12-04 23:21:27,090] [ WARNING] - PaddleHub v1.8 has deprecated the reader and feed_list parameters in the nlp Task. We provided an easier usage, in which you can use your tokenizer to preprocess dataset and run task in a clear flow. New demo see https://github.com/PaddlePaddle/PaddleHub/blob/release/v1.8/demo/text_classification/text_cls.py


In [ ]:
# finetune
run_states = cls_task.finetune_and_eval()

D:\rgzn\lib\site-packages\paddle\fluid\clip.py:779: UserWarning: Caution! 'set_gradient_clip' is not recommended and may be deprecated in future! We recommend a new strategy: set 'grad_clip' when initializing the 'optimizer'. This method can reduce the mistakes, please refer to documention of 'optimizer'.
  warnings.warn("Caution! 'set_gradient_clip' is not recommended "
[2020-12-04 23:21:41,921] [    INFO] - Strategy with warmup, linear decay, slanted triangle learning rate, weight decay regularization, 
D:\rgzn\lib\site-packages\paddle\fluid\executor.py:1093: UserWarning: There are no operators in the program to be executed. If you pass Program manually, please use fluid.program_guard to ensure the current Program is being used.
  warnings.warn(error_info)
[2020-12-04 23:21:42,040] [    INFO] - Try loading checkpoint from model\ckpt.meta
[2020-12-04 23:21:42,041] [    INFO] - PaddleHub model checkpoint not found, start from scratch...
[2020-12-04 23:21:42,565] [    INFO] - PaddleHub 

In [ ]:
# 预测
import numpy as np
    
inv_label_map = {val: key for key, val in reader.label_map.items()}

# Data to be prdicted
data = test[['content','comment_all']].fillna(' ').values.tolist()

run_states = cls_task.predict(data=data)
results = [run_state.run_results for run_state in run_states]


In [ ]:
# 生成预测结果

proba = np.vstack([r[0] for r in results])

submission = pd.DataFrame()
submission['id'] = test['id'].values
submission['ncw_prob_label_0c'], submission['fake_prob_label_0c'], submission['real_prob_label_0c'] = proba[:,0],proba[:,1],proba[:,2]

submission.to_csv('result/result.csv', index=False)
submission.head()
